In [ ]:
import argparse
import csv
from flax.training.common_utils import shard
from flax.jax_utils import replicate, unreplicate
import gc
import gzip
import jax
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from transformers import FlaxMarianMTModel, MarianTokenizer

In [2]:
import json

In [15]:
f = open('/data/pmd/dataset_coco.json')
data = json.load(f)

In [21]:
len(data['images'])

123287

In [19]:
data['images'][0]

{'filepath': 'val2014',
 'sentids': [770337, 771687, 772707, 776154, 781998],
 'filename': 'COCO_val2014_000000391895.jpg',
 'imgid': 0,
 'split': 'test',
 'sentences': [{'tokens': ['a',
    'man',
    'with',
    'a',
    'red',
    'helmet',
    'on',
    'a',
    'small',
    'moped',
    'on',
    'a',
    'dirt',
    'road'],
   'raw': 'A man with a red helmet on a small moped on a dirt road. ',
   'imgid': 0,
   'sentid': 770337},
  {'tokens': ['man',
    'riding',
    'a',
    'motor',
    'bike',
    'on',
    'a',
    'dirt',
    'road',
    'on',
    'the',
    'countryside'],
   'raw': 'Man riding a motor bike on a dirt road on the countryside.',
   'imgid': 0,
   'sentid': 771687},
  {'tokens': ['a',
    'man',
    'riding',
    'on',
    'the',
    'back',
    'of',
    'a',
    'motorcycle'],
   'raw': 'A man riding on the back of a motorcycle.',
   'imgid': 0,
   'sentid': 772707},
  {'tokens': ['a',
    'dirt',
    'path',
    'with',
    'a',
    'young',
    'person',

In [5]:
import glob
all_files = sorted(glob.glob('/data/pmd/data/*'))
red_caps = sorted(glob.glob('/data/annotations/*'))

In [6]:
sorted(red_caps)

['/data/annotations/abandoned_2017.json',
 '/data/annotations/abandoned_2018.json',
 '/data/annotations/abandoned_2019.json',
 '/data/annotations/abandoned_2020.json',
 '/data/annotations/abandonedporn_2017.json',
 '/data/annotations/abandonedporn_2018.json',
 '/data/annotations/abandonedporn_2019.json',
 '/data/annotations/abandonedporn_2020.json',
 '/data/annotations/absoluteunits_2018.json',
 '/data/annotations/absoluteunits_2019.json',
 '/data/annotations/absoluteunits_2020.json',
 '/data/annotations/airplants_2017.json',
 '/data/annotations/airplants_2018.json',
 '/data/annotations/airplants_2019.json',
 '/data/annotations/airplants_2020.json',
 '/data/annotations/alltheanimals_2019.json',
 '/data/annotations/alltheanimals_2020.json',
 '/data/annotations/amateurphotography_2017.json',
 '/data/annotations/amateurphotography_2018.json',
 '/data/annotations/amateurphotography_2019.json',
 '/data/annotations/amateurphotography_2020.json',
 '/data/annotations/amateurroomporn_2017.json'

In [3]:
num_lines = sum(1 for line in open('/data/pmd_indonesia/data/yfcc100m_subset.jsonl'))

In [4]:
num_lines

14824897

In [4]:
asf = []
for g in all_files:
    
    num_lines = sum(1 for line in open(g))
    asf.append(num_lines)

In [7]:
asf

[64885, 6588, 377673, 23362, 4321, 126277, 4167, 303036, 1278189, 95704]

In [72]:
for m in all_files:
    ls = []
    with open(m, "rb") as f:
        for i,line in enumerate(f):
            if line:
                example = json.loads(line)
                ls.append(example)

In [5]:
with open("/data/pmd_indonesia/data/yfcc100m_subset.jsonl", "r") as f:
    for i,line in enumerate(f):
        if i > 20:
            break
        if line:
            example = json.loads(line)
            print(example)

{'image_url': 'http://farm1.staticflickr.com/1/137943_1650c7cdc6.jpg', 'image': None, 'texts': ['Antena'], 'source': 'yfcc100m', 'meta': '{"photoid": 137943, "uid": "48600072071@N01", "unickname": "doctor+paradox", "usertags": "cameraphone,cayugaheights,green,hydrant,ithaca,moblog,nature,orange,tree,unfound", "downloadurl": "http://farm1.staticflickr.com/1/137943_1650c7cdc6.jpg", "licensename": "Attribution-NonCommercial-ShareAlike License", "licenseurl": "http://creativecommons.org/licenses/by-nc-sa/2.0/", "marker": 0, "title": "A+Picture+Share%21", "description": "Antenna"}'}
{'image_url': 'http://farm1.staticflickr.com/7/8053161_fda1675475.jpg', 'image': None, 'texts': ['Melihat ke arah cakrawala, dasar lembah tampak seperti padang rumput yang subur dengan bunga aster padat, tetapi melihat ke bawah, tanahnya sebagian besar adalah bebatuan, pasir, dan tanah gundul yang ditaburi tanaman di sana-sini. Tanaman Desert Gold setidaknya berjarak beberapa kaki dan berdiri setinggi paha. Mere

In [8]:
len(ls)

231647

In [ ]:
ls = []
from tqdm import tqdm
import ast
import os
with tqdm(total=os.path.getsize('yfcc100.jsonl')) as pbar:
    with open('yfcc100.jsonl', "r",167772160) as f:
        for line in f:
            example = ast.literal_eval(line)
            ls.append(example)
            pbar.update(len(line))

 57%|███████████▎        | 10747125650/18946883617 [27:57<20:51, 6551048.66it/s]

In [9]:
clean = []
import json
for batch in tqdm(ls):
    for g in batch:
        meta = json.dumps({
            "photoid":g['photoid'],
            "uid":g['uid'],
            "unickname":g['unickname'],
            "usertags":g['usertags'],
            "downloadurl":g['downloadurl'],
            "licensename":g['licensename'],
            "licenseurl":g['licenseurl'],
            "licenseurl":g['licenseurl'],
            "marker": g['marker'],
            "title": g['title'],
            "description": g['description']
        })
        dic = {
            "image_url": g['downloadurl'],
            "image": None,
            "texts": [g['description_clean']],
            "source": "yfcc100m",
            "meta": meta
        }
        clean.append(dic)

100%|██████████████████████████████████| 231647/231647 [03:57<00:00, 974.23it/s]


In [16]:
len(clean)

14824897

In [21]:
clean[56:59]

[{'image_url': 'http://farm1.staticflickr.com/25/46449131_dc653078f0.jpg',
  'image': None,
  'texts': ['Makanan dari seluruh dunia!!'],
  'source': 'yfcc100m',
  'meta': '{"photoid": 46449131, "uid": "28851385@N00", "unickname": "jordanfischer", "usertags": "id,institute+of+design,international+dinner", "downloadurl": "http://farm1.staticflickr.com/25/46449131_dc653078f0.jpg", "licensename": "Attribution License", "licenseurl": "http://creativecommons.org/licenses/by/2.0/", "marker": 0, "title": "ID+International+Dinner", "description": "Foods+from+around+the+world%21%21"}'},
 {'image_url': 'http://farm1.staticflickr.com/32/47976532_a039b93ac4.jpg',
  'image': None,
  'texts': ['Arembepe, Bahia (2004)'],
  'source': 'yfcc100m',
  'meta': '{"photoid": 47976532, "uid": "40905387@N00", "unickname": "martapiqs", "usertags": "arembepe,bahia,brasil,brazil,contest,surf,surfing", "downloadurl": "http://farm1.staticflickr.com/32/47976532_a039b93ac4.jpg", "licensename": "Attribution-NonCommerci

In [12]:
with open("yfcc100m_indo.jsonl", 'w') as f:
    for item in clean:
        f.write(json.dumps(item) + "\n")

In [35]:
clean[0:2]

[{'image_url': 'http://farm1.staticflickr.com/1/137943_1650c7cdc6.jpg',
  'image': None,
  'texts': ['Antena'],
  'source': 'yfcc100m',
  'meta': '{"photoid": 137943, "uid": "48600072071@N01", "unickname": "doctor+paradox", "usertags": "cameraphone,cayugaheights,green,hydrant,ithaca,moblog,nature,orange,tree,unfound", "downloadurl": "http://farm1.staticflickr.com/1/137943_1650c7cdc6.jpg", "licensename": "Attribution-NonCommercial-ShareAlike License", "licenseurl": "http://creativecommons.org/licenses/by-nc-sa/2.0/", "marker": 0, "title": "A+Picture+Share%21", "description": "Antenna"}'},
 {'image_url': 'http://farm1.staticflickr.com/7/8053161_fda1675475.jpg',
  'image': None,
  'texts': ['Melihat ke arah cakrawala, dasar lembah tampak seperti padang rumput yang subur dengan bunga aster padat, tetapi melihat ke bawah, tanahnya sebagian besar adalah bebatuan, pasir, dan tanah gundul yang ditaburi tanaman di sana-sini. Tanaman Desert Gold setidaknya berjarak beberapa kaki dan berdiri seti

In [24]:
ls_2 = []
import ast
import json
with open('yfcc100.jsonl', "r") as f:
    for i,line in enumerate(f):
        if i > 50:
            break
        if line:
            example = json.loads(line)
            ls_2.append(example)
            print(example)

{'image_url': 'https://multimedia-commons.s3-us-west-2.amazonaws.com/data/images/41d/d87/41dd8720fc6b7ffac962e3121dab33.jpg', 'image': None, 'texts': ['a link to a much higher resolution version, where you can actually see people without squinting:    Full Sized image  --'], 'source': 'yfcc100m', 'meta': '{\n  "photoid": 29060,\n  "uid": "34427466731@N01",\n  "title": "Christmas Panorama {notes}",\n  "description": "a link to a much higher resolution version, where you can actually see people without squinting:\\r\\n\\r\\n<a href=\\"http://photos1.flickr.com/29060_e90c2a0c1c_o.jpg\\">Full Sized image</a>\\r\\n--",\n  "usertags": "alberta,canada,christmas,edmonton,family,indoor,my \'good\' images,notes,panorama,photo,st albert,unfound,vacation",\n  "downloadurl": "http://farm1.staticflickr.com/1/29060_e90c2a0c1c.jpg",\n  "licensename": "Attribution License",\n  "licenseurl": "http://creativecommons.org/licenses/by/2.0/",\n  "marker": 0\n}'}
{'image_url': 'https://multimedia-commons.s3-u

In [24]:
ls[0:2]

[[{'photoid': 137943,
   'uid': '48600072071@N01',
   'unickname': 'doctor+paradox',
   'datetaken': '2004-08-01 18:13:06.0',
   'dateuploaded': '1091409186',
   'capturedevice': '',
   'title': 'A+Picture+Share%21',
   'description': 'Antenna',
   'usertags': 'cameraphone,cayugaheights,green,hydrant,ithaca,moblog,nature,orange,tree,unfound',
   'machinetags': '',
   'longitude': None,
   'latitude': None,
   'accuracy': None,
   'pageurl': 'http://www.flickr.com/photos/48600072071@N01/137943/',
   'downloadurl': 'http://farm1.staticflickr.com/1/137943_1650c7cdc6.jpg',
   'licensename': 'Attribution-NonCommercial-ShareAlike License',
   'licenseurl': 'http://creativecommons.org/licenses/by-nc-sa/2.0/',
   'serverid': 1,
   'farmid': 1,
   'secret': '1650c7cdc6',
   'secretoriginal': '1650c7cdc6',
   'ext': 'jpg',
   'marker': 0,
   'key': 'd29e7c6a3028418c64eb15e3cf577c2',
   'title_clean': 'Berbagi Gambar!',
   'description_clean': 'Antena'}],
 [{'photoid': 8053161,
   'uid': '5103574

In [7]:
import pandas as pd
df = pd.read_csv('/data/laion_indo_400m/part_4.tsv.gz',compression='gzip', sep='\t', header=0)

In [8]:
import datasets
ds = datasets.load_dataset('munggok/Laion_Indo',use_auth_token=True)

Using custom data configuration munggok--Laion_Indo-0e5252aa1cb5b9bc


KeyboardInterrupt: 

In [14]:
import csv 
import gzip
with gzip.open(open('/data/Laion_Indo/data/part_0.tsv.gz', "rb"), "rt", encoding="utf-8") as f:
    cf = csv.reader(f,delimiter="\t")
    next(cf, None) 
    for i,row in enumerate(cf):
        if i > 40:
            break
        print(row[1])
        

http://media.rightmove.co.uk/148k/147518/58718876/147518_SWO160154_EPCGRAPH_01_0000_max_135x100.png
https://thumbs.ebaystatic.com/images/g/DYEAAOSwMHdXR0Vh/s-l225.jpg
https://farm1.staticflickr.com/784/40182677504_27d67600f3_o.jpg
https://t2.ftcdn.net/jpg/00/58/35/35/240_F_58353522_3plS29kylx1KZQ0lU6pYHuCAhUINvCSp.jpg
https://findingblanche.files.wordpress.com/2013/07/photo4-1.jpg?w=764&
https://i.pinimg.com/236x/03/38/05/0338055833ee2afb2826fd3b7fd1dabc.jpg
http://d25hqtnqp5nl24.cloudfront.net/images/products/9/LN_364314_BP_9.jpg
http://cdn.pastemagazine.com/www/articles/2011/02/04/benaffleck.jpg?635339351410282708
https://chambermaster.blob.core.windows.net/images/members/653/1571/MemLogoSearch_MNDOT-new.jpg
https://cdn2.mecum.com/auctions/fl0121/fl0121-443116/thumbnails/dsc0164-1609793793911.jpg?1609799199000
http://images.fineartamerica.com/images-greeting-cards-real/greeting-card-clapton-live-david-lloyd-glover.jpg
https://buidln.clipdealer.com/001/841/344//search/1.jpg
https://ww

In [12]:
import datasets
ds_laion = datasets.load_dataset('/data/Laion_Indo/laion_indo.py')

Using custom data configuration default


Generating train split: 0 examples [00:00, ? examples/s]

Dataset laion_indo downloaded and prepared to /data/cache/laion_indo/default/0.0.0/a2f1b3ced124a4d05056f3bb2ef1d75a85234d4d74e4358e4bdbe841c78c251b. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [31]:
ds_laion['train']['image_url'][0]

'http://rlv.zcache.com/w_is_for_wagon_learn_to_spell_t_shirt-r5f1352dab76a4d40aa3370ba514f0bdd_f0cta_324.jpg'

In [1]:
import datasets

In [ ]:
ds = datasets.load_dataset('/data/pmd.py','laion_indo')

0 tables [00:00, ? tables/s]

Using custom data configuration munggok--Laion_Indo-5f17f96a1e14b2db


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Reusing dataset csv (/data/cache/munggok___csv/munggok--Laion_Indo-5f17f96a1e14b2db/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58)


Dataset csv downloaded and prepared to /data/cache/munggok___csv/munggok--Laion_Indo-5f17f96a1e14b2db/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58. Subsequent calls will reuse this data.


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
ds

In [8]:
!rm -rf cache/*

In [1]:
import datasets
ds = datasets.load_dataset('munggok/Laion_Indo')

Using custom data configuration munggok--Laion_Indo-5f17f96a1e14b2db
Reusing dataset csv (/data/cache/munggok___csv/munggok--Laion_Indo-5f17f96a1e14b2db/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58)


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
ds

In [5]:
ds['train'][100]

{'image_url': 'https://thumbs.dreamstime.com/t/blueberries-raspberries-ceramic-cup-assortment-berries-over-concrete-textured-background-top-view-composing-71976739.jpg',
 'image': None,
 'text': 'Blueberry dan raspberry. Cangkir keramik dengan ber',
 'source': '/data/Laion_Indo/laion_indo.py',
 'meta': '{\n  "metadata": "{\'sample_id\': \'2165647002180.0\', \'Height\': \'107.0\', \'Width\': \'160.0\', \'License\': \'?\', \'NSFW\': \'UNLIKELY\', \'Similarity\': \'0.32314419746398926\'}"\n}'}

In [6]:
ds['train']

Dataset({
    features: ['image_url', 'image', 'text', 'source', 'meta'],
    num_rows: 349934
})

In [6]:
data_urls = {}
_BASE_URL = "https://huggingface.co/datasets/munggok/mc4-id/resolve/main/mc4-id-filter/c4-id{split_suffix}.tfrecord-{index:05d}-of-{n_shards:05d}.json.gz"
_CONFIGS = {
    "tiny": {"train": 100, "validation": 1},
    "small": {"train": 250, "validation": 2},
    "medium": {"train": 500, "validation": 4},
    "large": {"train": 750, "validation": 6},
    "full": {"train": 1016, "validation": 8}
}
for split in ["train", "validation"]:
    data_urls[split] = [
        _BASE_URL.format(
            split_suffix="-validation" if split == "validation" else "",
            index=index,
            n_shards=8 if split == "validation" else 1024,
        )
        for index in range(_CONFIGS['full'][split])
    ]

In [7]:
data_urls['train']

['https://huggingface.co/datasets/munggok/mc4-id/resolve/main/mc4-id-filter/c4-id.tfrecord-00000-of-01024.json.gz',
 'https://huggingface.co/datasets/munggok/mc4-id/resolve/main/mc4-id-filter/c4-id.tfrecord-00001-of-01024.json.gz',
 'https://huggingface.co/datasets/munggok/mc4-id/resolve/main/mc4-id-filter/c4-id.tfrecord-00002-of-01024.json.gz',
 'https://huggingface.co/datasets/munggok/mc4-id/resolve/main/mc4-id-filter/c4-id.tfrecord-00003-of-01024.json.gz',
 'https://huggingface.co/datasets/munggok/mc4-id/resolve/main/mc4-id-filter/c4-id.tfrecord-00004-of-01024.json.gz',
 'https://huggingface.co/datasets/munggok/mc4-id/resolve/main/mc4-id-filter/c4-id.tfrecord-00005-of-01024.json.gz',
 'https://huggingface.co/datasets/munggok/mc4-id/resolve/main/mc4-id-filter/c4-id.tfrecord-00006-of-01024.json.gz',
 'https://huggingface.co/datasets/munggok/mc4-id/resolve/main/mc4-id-filter/c4-id.tfrecord-00007-of-01024.json.gz',
 'https://huggingface.co/datasets/munggok/mc4-id/resolve/main/mc4-id-fil

In [7]:
len(ls)

NameError: name 'ls' is not defined

In [8]:
BATCH_SIZE = 8

In [ ]:
ls = []
with open('/data/pmd/data/ln_open_images_test_captions_multi.jsonl', "rb") as f:
    for i,line in enumerate(f):
        if line:
            example = json.loads(line)
            ls.append(example)
for i in tqdm(range(0,len(ls[0:16]),BATCH_SIZE)):
    dataset_id = [m.get('dataset_id') for m in ls[i:i+BATCH_SIZE]]
    caption = [m.get('caption') for m in ls[i:i+BATCH_SIZE]]
    annotator_id = [m.get('annotator_id') for m in ls[i:i+BATCH_SIZE]]
    original_caption = [m.get('original_caption') for m in ls[i:i+BATCH_SIZE]]
    print(ls[i:i+8])

In [7]:
model = FlaxMarianMTModel.from_pretrained("Wikidepia/marian-nmt-enid", from_pt=True)

tokenizer = MarianTokenizer.from_pretrained("Wikidepia/marian-nmt-enid", source_lang="en")

2022-07-19 17:54:45.630288: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2022-07-19 17:54:46.385599: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2022-07-19 17:54:46.385706: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2022-07-19 17:54:46.385715: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make 

In [8]:
def generate(params, batch):
      output_ids = model.generate(batch["input_ids"], attention_mask=batch["attention_mask"], params=params, num_beams=4, max_length=64, early_stopping=True).sequences
      return output_ids

p_generate = jax.pmap(generate, "batch")

p_params = replicate(model.params)

In [9]:
def run_generate(input_str):
  inputs = tokenizer(input_str, return_tensors="jax", padding="max_length", truncation=True, max_length=64)
  p_inputs = shard(inputs.data)
  output_ids = p_generate(p_params, p_inputs)
  output_strings = tokenizer.batch_decode(output_ids.reshape(-1, 64), skip_special_tokens=True, max_length=64)
  return output_strings

In [11]:
f = open(red_caps[2])
  
# returns JSON object as 
# a dictionary
data = json.load(f)

In [16]:
info = data.get('info')


{'start_date': '2019-01-01',
 'end_date': '2019-12-31',
 'url': 'https://redcaps.xyz',
 'word_filter': {'num_removed': 7,
  'model': 'LDNOOBW/List-of-Dirty-Naughty-Obscene-and-Otherwise-Bad-Words'},
 'nsfw_filter': {'num_removed': 0, 'model': 'gantman/nsfw_model'},
 'face_filter': {'num_removed': 24,
  'model': 'redcaps-dataset/pytorch-retinaface',
  'confidence_threshold': 0.9},
 'version': '1.0'}

In [ ]:
def arrange_data_red_caps(dataset_id,caption,annotator_id,image_id,original_caption):
    try:
        lis_ = []
        caption_len = len(original_caption)
        if caption_len < 8 :
            dummy = ["tidak"] * (8 - caption_len)
            caption = caption + dummy
            original_caption = original_caption + dummy
            print("panjang original",len(original_caption),len(dummy))
            
        caption = run_generate(caption)[0:caption_len]
        original_caption = run_generate(original_caption)[0:caption_len]

        for dataset_id,caption,annotator_id,image_id,original_caption in zip(dataset_id,caption,annotator_id,image_id,original_caption):  # add other captions
                lis_.append({"dataset_id":dataset_id,"annotator_id":annotator_id,"image_id":image_id,"caption":caption,"original_caption":original_caption})

        gc.collect()
        return lis_

    except Exception as e:
        print(e)
        return

In [12]:
def arrange_data_pmd(dataset_id,caption,annotator_id,image_id,original_caption):
    try:
        lis_ = []
        caption_len = len(original_caption)
        if caption_len < 8 :
            dummy = ["tidak"] * (8 - caption_len)
            caption = caption + dummy
            original_caption = original_caption + dummy
            print("panjang original",len(original_caption),len(dummy))
            
        caption = run_generate(caption)[0:caption_len]
        original_caption = run_generate(original_caption)[0:caption_len]

        for dataset_id,caption,annotator_id,image_id,original_caption in zip(dataset_id,caption,annotator_id,image_id,original_caption):  # add other captions
                lis_.append({"dataset_id":dataset_id,"annotator_id":annotator_id,"image_id":image_id,"caption":caption,"original_caption":original_caption})

        gc.collect()
        return lis_

    except Exception as e:
        print(e)
        return

In [20]:
all_files[0].split('/')[-1]

'ln_ade20k_train_captions_multi.jsonl'

In [29]:
BATCH_SIZE = 1024
for file in enumerate(tqdm(all_files,desc='iterate fille',position = 0,leave=True)):
    #print(file[1])
    file_name = file[1].split('/')[-1]
    ls = []
    with open(file[1], "rb") as f:
        for i,line in enumerate(f):
            if line:
                example = json.loads(line)
                ls.append(example)
    for i in tqdm(range(0,len(ls),BATCH_SIZE),desc='Translating',position = 1,leave=True):
        dataset_id = [m.get('dataset_id') for m in ls[i:i+BATCH_SIZE]]
        caption = [m.get('caption') for m in ls[i:i+BATCH_SIZE]]
        annotator_id = [m.get('annotator_id') for m in ls[i:i+BATCH_SIZE]]
        original_caption = [m.get('original_caption') for m in ls[i:i+BATCH_SIZE]]
        image_id = [m.get('image_id') for m in ls[i:i+BATCH_SIZE]]
        output_batch = arrange_data_pmd(dataset_id,caption,annotator_id,image_id,original_caption)
        name = '/data/pmd_indonesia' + file_name
        with open(name, "a") as outfile:
            for batch in output_batch:
                json.dump(batch, outfile)
                outfile.write('\n')

iterate fille:   0%|                                     | 0/10 [01:15<?, ?it/s]

Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length.


TypeError: 'NoneType' object is not iterable

In [13]:
print(['12331'] * 2)

['12331', '12331']


In [14]:
len(ls[0:18])

NameError: name 'ls' is not defined

In [106]:
nilai = ls[0:18]

In [134]:
len(nilai)

18

In [167]:
import json
for i in tqdm(range(0,len(nilai),8)):
    dataset_id = [m.get('dataset_id') for m in nilai[i:i+BATCH_SIZE]]
    caption = [m.get('caption') for m in nilai[i:i+BATCH_SIZE]]
    annotator_id = [m.get('annotator_id') for m in nilai[i:i+BATCH_SIZE]]
    original_caption = [m.get('original_caption') for m in nilai[i:i+BATCH_SIZE]]
    image_id = [m.get('image_id') for m in nilai[i:i+BATCH_SIZE]]
    output_batch = arrange_data_pmd(dataset_id,caption,annotator_id,image_id,original_caption)
    with open("test.jsonl", "a") as outfile:
        for batch in output_batch:
            json.dump(batch, outfile)
            outfile.write('\n')

 67%|██████████████████████████████               | 2/3 [00:00<00:00,  2.82it/s]

panjang original 8 6


100%|█████████████████████████████████████████████| 3/3 [00:01<00:00,  2.91it/s]


In [70]:
print(ls[0:18])

[{'dataset_id': 'Flick30k', 'image_id': '4960617393', 'annotator_id': 43, 'caption': 'In this image we can see a person walking on the sidewalk.', 'original_caption': 'In this image we can see a person walking on the sidewalk. Also we can see vehicle. And we can see mesh fence. There is a tree trunk. On the right side we can see building with windows. Also we can see pole. In the background we can see trees. Also we can see branch with leaves on the right side.'}, {'dataset_id': 'Flick30k', 'image_id': '4960617393', 'annotator_id': 43, 'caption': 'Also we can see vehicle.', 'original_caption': 'In this image we can see a person walking on the sidewalk. Also we can see vehicle. And we can see mesh fence. There is a tree trunk. On the right side we can see building with windows. Also we can see pole. In the background we can see trees. Also we can see branch with leaves on the right side.'}, {'dataset_id': 'Flick30k', 'image_id': '4960617393', 'annotator_id': 43, 'caption': 'And we can s

In [161]:
from tqdm import tqdm

In [162]:
with open("test.jsonl", "rb") as f:
    for line in f:
        if line:
            try:
                example = json.loads(line)
            except Exception as e:
                print(e)

100%|█████████████████████████████████████████| 442/442 [01:16<00:00,  5.81it/s]


In [153]:
ass = [m.get('caption') for m in ls[0:8]]
mass = [m.get('original_caption') for m in ls[0:8]]

In [ ]:
ass

In [ ]:
mass

In [90]:
saya = [0,2,5,6,7,8,323,4,1,4,5,6,7,6,2,42,5]
for i in tqdm(range(0,len(saya),8)):
    print(saya[i:i+8])

100%|██████████████████████████████████████████| 3/3 [00:00<00:00, 35645.64it/s]

[0, 2, 5, 6, 7, 8, 323, 4]
[1, 4, 5, 6, 7, 6, 2, 42]
[5]


In [ ]:
ls[0:7]

In [15]:
str1 = ['saya','suka','kamu','sekali','saya','juga','tidak','ada']

In [16]:
run_generate(str1)

['saya', 'suka', 'kamu', 'sekali', 'saya', 'juga', 'tidak', 'ada']

In [ ]:
run_generate(mass)

In [ ]:
def arrange_data()

In [ ]:
def read_tsv_file(tsv_path):
    df = pd.read_parquet(tsv_path)
    print("Number of Examples:", df.shape[0], "for", tsv_path)
    return df

In [ ]:
def arrange_data(sample_id,url,text,height,width,license,nsfw,similarity):  # iterates through all the captions and save there translations
    try:
        lis_ = []

        output = run_generate(text)

        for sample_id,url,text,height,width,license,nsfw,similarity in zip(sample_id,url,output,height,width,license,nsfw,similarity):  # add other captions
                lis_.append({"SAMPLE_ID":sample_id,"URL":url,"TEXT":text,"HEIGHT":height,"WIDTH":width,"LICENSE":license,"NSFW":nsfw,"similarity":similarity})

        gc.collect()
        return lis_

    except Exception as e:
        print(e)
        return

In [ ]:
_df = read_tsv_file('/data/laion400m/part-00000-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet')

In [ ]:
df = _df[_df['TEXT'].notna()]

In [ ]:
df['TEXT'].isnull().values.any()

In [ ]:
_df.isnull().values.any()

In [ ]:
train_df = _df

In [ ]:
train_df

In [ ]:
train_df['similarity']

In [ ]:
df = train_df
output_file_name = os.path.join('/data/laion_indo_400m', "part_1.tsv")
with open(output_file_name, 'w', newline='') as outtsv:  # creates a blank tsv with headers (overwrites existing file)
    writer = csv.writer(outtsv, delimiter='\t')
    writer.writerow(["SAMPLE_ID","URL","TEXT","HEIGHT","WIDTH","LICENSE","NSFW","similarity"])

In [ ]:
len(df)

In [ ]:
BATCH_SIZE = 1024

In [ ]:
for i in tqdm(range(0,len(df),BATCH_SIZE)):
    output_batch = arrange_data(list(df["SAMPLE_ID"])[i:i+BATCH_SIZE], list(df["URL"])[i:i+BATCH_SIZE], list(df["TEXT"])[i:i+BATCH_SIZE],list(df["HEIGHT"])[i:i+BATCH_SIZE],list(df["WIDTH"])[i:i+BATCH_SIZE],list(df["LICENSE"])[i:i+BATCH_SIZE],list(df["NSFW"])[i:i+BATCH_SIZE],list(df["similarity"])[i:i+BATCH_SIZE])
    with open(output_file_name, "a", newline='') as f:
      writer = csv.DictWriter(f, fieldnames=["SAMPLE_ID","URL","TEXT","HEIGHT","WIDTH","LICENSE","NSFW","similarity"], delimiter='\t')
      for batch in output_batch:
          writer.writerow(batch)